### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
#rename df
comp_data_df = school_data_complete
#comp_data_df.head()  test output 

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
tot_school_cnt = comp_data_df["school_name"].nunique()
#print("tot # schools:"+str(tot_school_cnt))
tot_student_cnt = comp_data_df["Student ID"].nunique()   # some students have same name, so count the unique student IDs.
tot_budget = school_data["budget"].sum()
avg_math = student_data["math_score"].mean()
avg_reading = student_data["reading_score"].mean()
# % students passed (70% or greater = passing) 
num_passed_math = len(student_data.loc[(student_data["math_score"]>=70)])
pct_passed_math = (num_passed_math/tot_student_cnt)*100
num_passed_reading = len(student_data.loc[(student_data["reading_score"]>=70)])
pct_passed_reading = (num_passed_reading/tot_student_cnt)*100

num_passed_both = len(student_data.loc[((student_data["math_score"]>=70) & (student_data["reading_score"]>=70))])
pct_passed_both = (num_passed_both/tot_student_cnt)*100

summary_df = pd.DataFrame({"Total Schools": [tot_school_cnt],
                           "Total Students": [tot_student_cnt],
                           "Total School Budget": [tot_budget],
                           "Average Math Score": avg_math,
                           "Average Reading Score": avg_reading,
                           "% Passing Math": pct_passed_math,
                          "% Passing Reading": pct_passed_reading,
                          "% Overall Passing": pct_passed_both})
summary_df.style.format({"Total Students":"{:,}","Total School Budget":"${:,.2f}"})

,Total Schools,Total Students,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
school_groupby = student_data.groupby("school_name")
#school_groupby["Student ID"].count()
stype = school_data[["school_name","type","budget"]]
#print(school_data)
#print(stype.dtypes)

school_summ_df = comp_data_df["school_name"].value_counts().to_frame().reset_index().rename(columns = {"index":"school_name","school_name":"Total Students"})


school_summ_df = pd.merge(school_summ_df, stype, how="left", on=["school_name", "school_name"]).rename(columns = {"type":"School Type","budget":"Total School Budget"})
school_summ_df = school_summ_df.sort_values(by=["school_name"])
school_summ_df["Per Student Budget"] =school_summ_df["Total School Budget"]/school_summ_df["Total Students"] 

#school_summ_df  test output to ensure this cell works as expected


In [4]:
# create avgs and merge into summary
avgs_df = school_groupby.mean().reset_index().rename(columns = {"reading_score":"Avg Reading Score","math_score":"Avg Math Score"})
#test msg print(avgs_df)

#remove unneeded column
del avgs_df["Student ID"]
school_summ_df = pd.merge(school_summ_df, avgs_df, how="left", on=["school_name", "school_name"])
#school_summ_df   test output to ensure cell works as expected

In [5]:
# update the index to be the school name   (todo - double check above code if can remove the reset index...and then don't need this)
school_summ_df= school_summ_df.set_index("school_name")  


In [6]:
#add new columns, init to 0
school_summ_df["% Passing Math"]=0.000000   #6 decimal places to match output of example 
school_summ_df["% Passing Reading"]=0.000000  
school_summ_df["% Overall Passing"]=0.000000 

In [7]:
# figure out % passing per school for math/reading/both

#student_data  test output
#print(school_summ_df.index.values.tolist())  test output

# test statement to see if can get the # of students passed math
#num_passed_math = len(student_data.loc[((student_data["school_name"]=="Bailey High School")&(student_data["math_score"]>=70))])

for school in school_summ_df.index.values.tolist():  #todo  update this to be the real  df instead of test
    #testingHelp: print("in loop "+school+" #students:" + str(school_summ_df.loc[school,"Total Students"]))
    ## calc math %
    num_passed_math = len(student_data.loc[((student_data["school_name"]==school)&(student_data["math_score"]>=70))])
    school_summ_df.at[school,"% Passing Math"]= (num_passed_math/school_summ_df.loc[school,"Total Students"])*100 
    ## calc reading %
    num_passed_reading = len(student_data.loc[((student_data["school_name"]==school)&(student_data["reading_score"]>=70))])
    school_summ_df.at[school,"% Passing Reading"]= (num_passed_reading/school_summ_df.loc[school,"Total Students"])*100 
    ## calc overall %
    num_passed_reading = len(student_data.loc[((student_data["school_name"]==school)&(student_data["math_score"]>=70)&(student_data["reading_score"]>=70))])
    school_summ_df.at[school,"% Overall Passing"]= (num_passed_reading/school_summ_df.loc[school,"Total Students"])*100 

In [8]:
display_school_summ = school_summ_df
display_school_summ.index.name=None #removing school_name index to match output in instructions

#added column list in below statement to display them in the exact order displayed in the instructions
#formatted the $ columns to add $, commas, & display 2 decimal places
display_school_summ[["School Type","Total Students","Total School Budget","Per Student Budget",
                     "Avg Math Score", "Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].style.format({"Per Student Budget":"${:,.2f}","Total School Budget":"${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [9]:
#print(school_summ_df.sort_values(by=["% Overall Passing"] ascending = False).head(5)  #could use print, or just list what you want the notebook to print like below statement.   below statement looks nicer
top_schools = school_summ_df.sort_values(by=["% Overall Passing"], ascending = False).head(5)
top_schools.index.name=None #removing school_name index to match output in instructions

#display in order that matches the instructions. Added formatting to $ columns 
top_schools[["School Type","Total Students","Total School Budget","Per Student Budget","Avg Math Score", 
             "Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].style.format({"Per Student Budget":"${:,.2f}","Total School Budget":"${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [10]:
#print(school_summ_df.sort_values(by=["% Overall Passing"]).head(5))  #could use print, or just list what you want the notebook to print like below statement.   below statement looks nicer
bottom_schools=school_summ_df.sort_values(by=["% Overall Passing"]).head(5)
bottom_schools.index.name=None #removing school_name index to match output in instructions

#display in order that matches the instructions added formatting to $ columns 
bottom_schools[["School Type","Total Students","Total School Budget","Per Student Budget","Avg Math Score",
                "Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"]].style.format({"Per Student Budget":"${:,.2f}","Total School Budget":"${:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [11]:
#group by name+grade
grade_groupby = student_data.groupby(["school_name","grade"])
avgs_by_grade_df = grade_groupby.mean().rename(columns = {"reading_score":"Avg Reading Score","math_score":"Avg Math Score"})
del avgs_by_grade_df["Student ID"]

#below are testing outputs to ensure cell was working as expected.
#avgs_by_grade_df

In [12]:
avgs_by_grade_df.loc["Bailey High School","9th"] # this is a series
# avgs_by_grade_df.filter(like="9th",axis=0)
reading_9th_avgs= avgs_by_grade_df.filter(like="9th",axis=0)["Avg Reading Score"]
math_9th_avgs = avgs_by_grade_df.filter(like="9th",axis=0)["Avg Math Score"]

reading_10th_avgs= avgs_by_grade_df.filter(like="10th",axis=0)["Avg Reading Score"]
math_10th_avgs = avgs_by_grade_df.filter(like="10th",axis=0)["Avg Math Score"]

reading_11th_avgs= avgs_by_grade_df.filter(like="11th",axis=0)["Avg Reading Score"]
math_11th_avgs = avgs_by_grade_df.filter(like="11th",axis=0)["Avg Math Score"]

reading_12th_avgs= avgs_by_grade_df.filter(like="12th",axis=0)["Avg Reading Score"]
math_12th_avgs = avgs_by_grade_df.filter(like="12th",axis=0)["Avg Math Score"]

#below are testing outputs to ensure cell was working as expected.
#math_12th_avgs
#math_9th_avgs

In [13]:
#coding math scores by school
# remove unnecessary "grade" index and rename the column to be grade value
math_9th_avgs = math_9th_avgs.reset_index(level="grade").rename(columns = {"Avg Math Score":"9th"})
math_10th_avgs = math_10th_avgs.reset_index(level="grade").rename(columns = {"Avg Math Score":"10th"}) 
math_11th_avgs = math_11th_avgs.reset_index(level="grade").rename(columns = {"Avg Math Score":"11th"})
math_12th_avgs = math_12th_avgs.reset_index(level="grade").rename(columns = {"Avg Math Score":"12th"})
del math_9th_avgs["grade"]
del math_10th_avgs["grade"]
del math_11th_avgs["grade"]
del math_12th_avgs["grade"]


In [14]:
# concat series-es into summary dataframe
summ_math_by_school_df = pd.concat([math_9th_avgs, math_10th_avgs, math_11th_avgs, math_12th_avgs], axis=1)
summ_math_by_school_df.index.name=None  #to match the instructions, removing the index
summ_math_by_school_df

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [15]:
# remove unnecessary "grade" index and rename the column to be grade value
reading_9th_avgs = reading_9th_avgs.reset_index(level="grade").rename(columns = {"Avg Reading Score":"9th"})
reading_10th_avgs = reading_10th_avgs.reset_index(level="grade").rename(columns = {"Avg Reading Score":"10th"}) 
reading_11th_avgs = reading_11th_avgs.reset_index(level="grade").rename(columns = {"Avg Reading Score":"11th"})
reading_12th_avgs = reading_12th_avgs.reset_index(level="grade").rename(columns = {"Avg Reading Score":"12th"})
del reading_9th_avgs["grade"]
del reading_10th_avgs["grade"]
del reading_11th_avgs["grade"]
del reading_12th_avgs["grade"]

#reading_9th_avgs  testing output to be sure cells code worked as expected

In [16]:
#put into frame
summ_read_by_school_df = pd.concat([reading_9th_avgs, reading_10th_avgs, reading_11th_avgs, reading_12th_avgs], axis=1)
summ_read_by_school_df  #this shows an index name "school_name"
summ_read_by_school_df.index.name=None  #to match the instructions, removing the index
summ_read_by_school_df


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [17]:
# use bins here
school_summ_df.sort_values(by='Per Student Budget')  #used to view the data and come up with bins
bins = [0, 585,630,645,680]
b_labels = ["<$585","$585-630","$630-645","$645-680"]

In [18]:
bud_bined_df=pd.cut(school_summ_df["Per Student Budget"], bins, labels=b_labels)
#bud_bined_df  testing output
#add binned column back to summary df
school_summ_df["Spending Ranges (Per Student)"] = bud_bined_df
bud_groupby = school_summ_df.groupby("Spending Ranges (Per Student)")
#display averages by binned column
#adding a round to this statement so it matches the example given in the instructions
bud_groupby["Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean().round(2)


C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
$645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [19]:
#create size bins/size labels
s_bins = [0, 1000,2000,5000]
sb_labels = ["Small(<1000)","Medium(1000-2000)","Medium(2000-5000)"]
#bin data
numStud_bined_df=pd.cut(school_summ_df["Total Students"], s_bins, labels=sb_labels)
#add binned data to summary df
school_summ_df["School Size"] = numStud_bined_df
#group by the binned column and display the avg per bin
numStud_groupby = school_summ_df.groupby("School Size")
numStud_groupby["Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()

C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # Remove the CWD from sys.path while we load stuff.


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium(1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Medium(2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [20]:
# use groupby as was done for school name

stype_groupby = school_summ_df.groupby("School Type")
stype_groupby["Avg Math Score","Avg Reading Score","% Passing Math","% Passing Reading","% Overall Passing"].mean()

C:\Users\kunaual\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.


,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
